In [1]:
import unicodedata
import string
import re
import random
import time
import datetime
import math
import socket
hostname = socket.gethostname()

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
from masked_cross_entropy import *

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
%matplotlib inline


emb = nn.Embedding(10, 8)
print(emb.weight)
input_lengths = [3,3]
inp_seq = Variable(torch.LongTensor([[0,1],[1,2],[2,3]])) # max_length = 3, batch size = 2 (basically 2 sentences of size 3 each)
embedded = emb(inp_seq)
print(embedded)

Parameter containing:
-1.0621 -0.3288  1.1981  0.2501 -1.6450  0.2469 -0.8459 -1.3823
-0.4694  1.9801  1.6282 -1.3543  0.2387  1.7283 -0.7127 -0.5819
 0.5085  2.1128 -1.7126 -1.2009 -0.5804 -1.6153 -0.0525 -0.1961
-0.5935 -0.3429 -0.2120  1.9721  0.2144 -1.2680 -1.0744  1.3904
-0.4951 -0.6379  0.3515  1.2364 -0.0588 -0.7529 -0.7056  2.1758
-1.9269  0.1559  1.7837  1.1505  0.8843 -1.3509  0.4502 -0.7376
-2.3874 -0.9906  0.8133  0.2321 -1.5676  0.4423 -0.6881 -1.1063
 0.3793  1.0939 -0.7279  0.8056 -0.7481 -0.1250  1.6187 -0.4917
 0.1322  0.3280  0.2431 -1.0077  0.4445 -1.5049 -1.6693 -1.7160
 1.1499  1.3409  0.4392  0.1849  2.5987 -1.4353 -0.2236  0.6242
[torch.FloatTensor of size 10x8]

Variable containing:
(0 ,.,.) = 
 -1.0621 -0.3288  1.1981  0.2501 -1.6450  0.2469 -0.8459 -1.3823
 -0.4694  1.9801  1.6282 -1.3543  0.2387  1.7283 -0.7127 -0.5819

(1 ,.,.) = 
 -0.4694  1.9801  1.6282 -1.3543  0.2387  1.7283 -0.7127 -0.5819
  0.5085  2.1128 -1.7126 -1.2009 -0.5804 -1.6153 -0.0525 -0.196

In [2]:
packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
packed
gru = nn.GRU(8,8,1,bidirectional=True)
outputs, hidden = gru(packed, None)

In [3]:
outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs)
print(outputs, "\n\n\n", hidden)

Variable containing:
(0 ,.,.) = 

Columns 0 to 8 
  -0.3887  0.2049 -0.1395  0.2054  0.0556 -0.0030 -0.2052  0.1519  0.1865
 -0.2024 -0.1286 -0.4030 -0.0180  0.2031 -0.2357  0.4924  0.4134  0.1953

Columns 9 to 15 
   0.1340  0.3503 -0.2416  0.1559 -0.0524 -0.3081 -0.1229
  0.1530  0.5486 -0.6176  0.2426 -0.1266 -0.2613  0.0365

(1 ,.,.) = 

Columns 0 to 8 
  -0.3909 -0.0164 -0.4796  0.1508  0.2107 -0.2635  0.4387  0.4684  0.2760
 -0.2676  0.0696 -0.1210  0.1447  0.1245  0.6422  0.5266 -0.2010 -0.0368

Columns 9 to 15 
   0.1327  0.4902 -0.6126  0.2420 -0.0181 -0.1562  0.0288
  0.4322  0.2652 -0.4692 -0.3856 -0.3003 -0.0333 -0.1828

(2 ,.,.) = 

Columns 0 to 8 
  -0.3636  0.1512 -0.1533  0.2590  0.1185  0.6338  0.5081 -0.2449  0.1379
 -0.2681  0.3872 -0.2309  0.3077  0.2475  0.4564 -0.1219 -0.1519 -0.3153

Columns 9 to 15 
   0.4016  0.0483 -0.4449 -0.4408 -0.1412  0.1036 -0.1091
  0.0497  0.3328  0.0650  0.0550 -0.1937 -0.1735 -0.2291
[torch.FloatTensor of size 3x2x16]
 


 Variable c

In [4]:
encoder_hidden = hidden[:,1]
encoder_hidden

Variable containing:
-0.2681  0.3872 -0.2309  0.3077  0.2475  0.4564 -0.1219 -0.1519
 0.1953  0.1530  0.5486 -0.6176  0.2426 -0.1266 -0.2613  0.0365
[torch.FloatTensor of size 2x8]

In [5]:
x = torch.randn(3, 10, 8) # (num_layers*num_directions, batch, hidden_size)
x


(0 ,.,.) = 
  1.1649  0.6227  0.0377  1.5082  1.2079 -1.0263  0.9527 -1.0573
  0.9981  1.2160 -1.6219  1.4166  0.4511  0.2284  0.1482  0.9703
 -0.0821 -1.9819  1.2165  0.6273  1.2947 -0.1499 -0.3012  1.3003
  0.1721  1.6277 -0.9819  0.2184 -1.3980  0.9430  0.4502  0.8252
 -0.0762 -0.8091  0.3428  0.8362  0.1281 -1.2115 -0.6643 -0.2419
  0.2324  0.2410 -0.1980  0.4796 -0.3987  0.4532  0.7103 -1.4057
  0.0390 -1.3084 -1.6195  0.9308  0.4632 -1.2675 -0.5356 -0.1859
  0.9190  0.1293 -0.5696 -1.1240  1.0174 -1.9136  1.7826  0.6272
  0.4495  0.1624  1.4156 -0.1969 -0.8393  0.2471  0.2182  0.7933
 -1.5586 -0.5604 -0.1043 -0.7584  0.8421 -0.3171 -0.3432  0.6025

(1 ,.,.) = 
  0.2929 -0.8798  1.0703  0.7293  0.0651  0.2613  0.1121  2.6345
 -0.1835  0.8912  0.8478 -0.9610 -0.2272  0.8357 -0.7404 -1.1439
 -0.6491  0.6822 -0.0611  0.6637  1.1973  0.0192  0.3271  0.1196
 -1.0026  0.4247 -0.1238 -1.5490 -0.4546  1.7611 -0.3966 -0.2024
 -0.4742  1.0550  1.0576  0.2324 -1.3139  1.7602 -0.2670  1.8933

In [6]:
attn = nn.Linear(8 * 2, 8)
v = nn.Parameter(torch.FloatTensor(1, 8))

In [8]:
encoder_hidden

Variable containing:
-0.1200  0.1529  0.5433  0.1439 -0.3919  0.3840  0.3570  0.6396
-0.1981  0.4209  0.0659  0.0410  0.3421 -0.1809 -0.0197 -0.3295
[torch.FloatTensor of size 2x8]

In [9]:
hid = encoder_hidden[0,:]
hid

Variable containing:
-0.1200
 0.1529
 0.5433
 0.1439
-0.3919
 0.3840
 0.3570
 0.6396
[torch.FloatTensor of size 8]

In [10]:
enc_out = x[0,0]
enc_out


 1.0613
-0.5589
-1.9943
 1.0971
 0.1027
 1.4603
 0.2676
-0.6679
[torch.FloatTensor of size 8]

In [11]:
cated = torch.cat((hid, enc_out), 0)
cated

Variable containing:
-0.1200
 0.1529
 0.5433
 0.1439
-0.3919
 0.3840
 0.3570
 0.6396
 1.0613
-0.5589
-1.9943
 1.0971
 0.1027
 1.4603
 0.2676
-0.6679
[torch.FloatTensor of size 16]

In [12]:
(v.squeeze(0)).dot(attn(cated))

Variable containing:
 2.7114e+18
[torch.FloatTensor of size 1]

In [13]:
attn(cated)

Variable containing:
 0.3068
 0.6808
 0.0503
-0.7791
 0.4188
-0.2608
-0.0225
 0.8814
[torch.FloatTensor of size 8]

In [13]:
temp = Variable(torch.randn(1,15))
topv, topi = temp.data.topk(5)
temp

Variable containing:

Columns 0 to 9 
 1.4427 -1.8166  0.5621 -0.8192  1.8608  1.6298 -2.0861  1.0015  0.0954 -0.9956

Columns 10 to 14 
 0.0110  0.7718  0.6203  0.5950  1.0942
[torch.FloatTensor of size 1x15]

In [15]:
topi[0].numpy()

array([ 4,  5,  0, 14,  7], dtype=int64)

# Beam search trails

In [37]:
# The vocab size = 10 (assume)
vocab_size = 20
k = 10
index_of_start = 0
index_of_end = 19

scores = torch.FloatTensor(vocab_size).zero_()
scores

prevKs = []
nextYs = [torch.LongTensor(vocab_size).fill_(0)]
nextYs[0][0] = index_of_start

In [ ]:
def update_dictionary(target_sequence, topv, topi, key, dec_hidden):
    if len(target_sequence) == 0:
        for i in range(topi):
            target_sequence.update({str(topi[i]) : [topv[i], dec_hidden] })
        else:
            prev_val = target_sequence[key][0]
            target_sequence.update({key+"-"+str(topi[i]) : [topv[i]*prev_val, dec_hidden] })
            
kmax = 15
    
# Run through decoder
target_sequence = dict()

for di in range(max_length):

    if di == 0:
        decoder_output, decoder_hidden, decoder_attention = decoder( decoder_input, decoder_hidden, encoder_outputs )
        topv, topi = decoder_output.data.topk(kmax)
        topv = topv[0].numpy()
        topi = topi[0].numpy()
        update_dictionary(target_sequence, topv, topi, None, decoder_hidden)
    else:
        temp = target_sequence.copy()
        keys = list(temp.keys())
        for i in range(len(keys)):
            inp = keys[i].split("-")[-1] if len(x) > 1 else keys[i]
            if inp != EOS_token:
                dec_input = Variable(torch.LongTensor([inp]))
                decoder_output, dec_hidden, decoder_attention = decoder( dec_input, temp[keys[i]][1], encoder_outputs )
                topv, topi = decoder_output.data.topk(kmax)
                topv = topv[0].numpy()
                topi = topi[0].numpy()
                update_dictionary(target_sequence, topv, topi, keys[i], dec_hidden)
    
    
    # Sort the target_Sequence dictionary to keep top 15 sequences only
    target_sequence = dict(sorted(target_sequence.items(), key=lambda x: x[1][0], reverse=True)[:kmax])
    
    
    
    # Get the sequence with maximum probability
    seq = sorted(target_sequence.items(), key=lambda x: x[1][0], reverse=True)[:1][0][0]
    
    # Get the decoded words:
    decoded_words_indices = seq.split("-")
    decoded_words = [output_lang.index2word(int(i)) for i in decoded_words_indices]
    
    topv, topi = decoder_output.data.topk(1)
    ni = topi[0][0]
    if ni == EOS_token:
        decoded_words.append('<EOS>')
        break
    else:
        decoded_words.append(output_lang.index2word[ni])

    # Next input is chosen word
    decoder_input = Variable(torch.LongTensor([ni]))
    if USE_CUDA: decoder_input = decoder_input.cuda()

# Set back to training mode
encoder.train(True)
decoder.train(True)

return decoded_words, decoder_attentions[:di+1, :len(encoder_outputs)]

In [124]:
tt = dict()
tt.update({"1-2":[1,2]})
tt.update({"3-4":[3,20]})
tt.update({"4-5":[5,21]})
tt.update({"5-6":[7,29]})
tt.update({"6-7":[9,22]})
tt.update({"7-8-9-19-20":[11,25]})
tt

{'1-2': [1, 2],
 '3-4': [3, 20],
 '4-5': [5, 21],
 '5-6': [7, 29],
 '6-7': [9, 22],
 '7-8-9-19-20': [11, 25]}

In [129]:
seq = sorted(tt.items(), key=lambda x: x[1][0], reverse=True)[:1][0][0]
seq

'7-8-9-19-20'

In [133]:
dec = seq.split("-")
dec = [output_lang.index2word(int(i)) for i in dec]
dec

[7, 8, 9, 19, 20]